# Contextual data

Once you've identified areas of interest where multiple datasets intersect, you can pull additional data to provide further context. For example:

1. landcover 
2. global elevation data 

In [1]:
import coincident
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from scipy.stats import median_abs_deviation

%matplotlib inline

## Identify a primary dataset

Start by loading a full resolution polygon of a 3DEP LiDAR workunit which has a known start_datetime and end_datatime:

In [ ]:
workunit = "CO_WestCentral_2019"
df_wesm = coincident.search.wesm.read_wesm_csv()
gf_lidar = coincident.search.wesm.load_by_fid(
    df_wesm[df_wesm.workunit == workunit].index
)

gf_lidar

## Load raster datasets with Xarray

In [3]:
search_aoi = gf_lidar.simplify(0.01)

In [4]:
gf_wc = coincident.search.search(
    dataset="worldcover",
    intersects=search_aoi,
    # worldcover is just 2020 an 2021, so pick one
    datetime=["2020"],
)  # Asset of interest = 'map'

In [5]:
gf_cop30 = coincident.search.search(
    dataset="cop30",
    intersects=search_aoi,
)  # Asset of interest = 'data'

In [6]:
ds = coincident.io.xarray.to_dataset(
    gf_cop30,
    aoi=search_aoi,
    # chunks=dict(x=2048, y=2048), # manual chunks
    resolution=0.00081,  # ~90m
    mask=True,
).compute()

In [ ]:
ds

In [8]:
ds = ds.rename(data="elevation")

In [ ]:
ds.elevation.isel(time=0).plot.imshow();

In [ ]:
# Same with LandCover
ds = coincident.io.xarray.to_dataset(
    gf_wc,
    bands=["map"],
    aoi=search_aoi,
    mask=True,
    # resolution=0.00027, #~30m
    resolution=0.00081,  # ~90m
)
ds

In [11]:
ds = ds.rename(map="landcover")

In [12]:
ds = ds.compute()

In [ ]:
ax = coincident.plot.plot_esa_worldcover(ds)
ax.set_title("ESA WorldCover");